# CEO-Request Challenge

## 1. Summary of Problem Statement

❓ **How could Olist improve its profit** ❓

### P&L Rules

#### Revenues  
**Sales fees:** Olist takes a **10% cut** on the product price (excl. freight) of each order delivered  
**Subscription fees:** Olist charges **80 BRL by month** per seller

#### Costs
**Reputation costs** *estimated* per order with bad reviews (<= 3 stars)  

💡 In the long term, bad customer experience has business implications: low repeat rate, immediate customer support cost, refunds or unfavorable word of mouth communication. We will assume that we have an estimate measure of the monetary cost for each bad review:
```python
# review_score: cost(BRL)
{'1 star': 100
'2 stars': 50
'3 stars': 40
'4 stars': 0
'5 stars': 0}
```

**IT costs:** Olist's **total cumulated IT Costs** scale with the square root of the total number of sellers that has ever join the platform, as well as the square root of the total cumulated number of products that was ever sold.  

$IT\_costs = \alpha * \sqrt{n\_sellers} + \beta * \sqrt{n\_products}$  
Olist's data team gave us the following values for these scaling parameters:
- $\alpha = 3157.27$
- $\beta = 978.23$

💡 Both number of sellers to manage and sales transaction are costly for IT systems.  
💡 Yet square roots reflect scale-effects: IT-system are often more efficient as they grow bigger.  
💡 Alpha > Beta means that "Olist has a lower IT Cost with few sellers selling a lot of products rather than the opposite  
- with **1000 sellers** and a total of **100 products** sold, the total IT cost accumulates to 109,624 BRL
- with **100 sellers** and a total of **1000 products** sold, the total IT cost accumulates to 62,507 BRL

Finally, The IT department also told you that since the birth of the marketplace, cumulated IT costs have amounted to **500,000 BRL**.

### Key Findings, so far

- `wait_time` is the most significant factor behind low review scores  
- `wait_time` is made up of seller's `delay_to_carrier` + `carrier_delivery_time`.  
- The latter being outside of Olist's direct control, improving it is not a quick-win recommendation
- On the contrary, a better selection of `sellers` can positively impact the `delay_to_carrier` and reduce the number of bad `review_scores` on Olist.
- Comments of the bad reviews showed that some were linked to the seller or to the product itself.

💡 We recommend you to start with the the guided seller analysis in part 2 below  
💪 But feel free to investigate into other hypothesis instead with part 3

## 2. Should Olist remove under-performing sellers from its marketplace? 🕵🏻
*(recommended)*

To analyze the impact of removing the worse sellers from Olist's marketplace, we will perform a **what-if analysis**

👉 **What would have happened if Olist had never accepted these sellers in the first place?**  

*(In practice, it's hard to know in advance who is a good seller, but let's start with this approach and iterate later).*

### 2.1 Data Preparation

Compute, for each `seller_id`, and cumulated since the beginning:
- the `revenues` it brings
- the `review_costs` associated with all its bad reviews
- the resulting `profits` (revenues - costs)

👉 Write down a step-by-step strategy to create the DataFrame you need


⚠️ Don't start from scratch, update your existing package 😉

Starting from the `Seller` class of your `olist` package:

Edit the `get_training_data` method so that the DataFrame it returns contains the fields:
- `revenues`: sum of subscription and sales fees revenues
- `cost_of_reviews`: sum of costs associated with bad reviews
- `profits`: `revenues` - `cost_of_reviews`

#### Data Prep

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import math

In [3]:
from olist.seller import Seller

seller = Seller()
sellers = seller.get_training_data()
sellers.head(5)

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales,share_of_one_stars,share_of_five_stars,review_score
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000,218.70,0.333333,0.333333,3.00
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025,11703.07,0.050000,0.725000,4.55
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,1,1,1.000,158.00,0.000000,1.000000,5.00
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.0,1,1,1.000,79.99,0.000000,1.000000,5.00
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,0.0,1,1,1.000,167.99,1.000000,0.000000,1.00


#### Revenues

In [4]:
#subscription = 80 * months on olist column 
#sales fees revenue = sales * 0.1
sellers['revenues'] = ((sellers.loc[:, 'months_on_olist'] * 80) + (sellers.loc[:, 'sales'] * 0.1))
sellers.head(5)

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales,share_of_one_stars,share_of_five_stars,review_score,revenues
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000,218.70,0.333333,0.333333,3.00,341.870
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025,11703.07,0.050000,0.725000,4.55,2290.307
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,1,1,1.000,158.00,0.000000,1.000000,5.00,15.800
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.0,1,1,1.000,79.99,0.000000,1.000000,5.00,7.999
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,0.0,1,1,1.000,167.99,1.000000,0.000000,1.00,16.799


#### Cost of Reviews

In [24]:
# sellers['cost_of_reviews'] = sellers.loc[:, 'review_score'].apply(lambda x: 100 if 1 <= x < 2 else (50 if 2 <= x < 3 else (40 if 3 <= x < 4 else 0)))

In [5]:
#creating the merged df to get the non-mean review score.
from olist.data import Olist
from olist.order import Order
olist = Olist()
data = olist.get_data()
order = Order()

orders_reviews = order.get_review_score()
orders_sellers = data['order_items'][['order_id', 'seller_id']].drop_duplicates()
df = orders_sellers.merge(orders_reviews, on='order_id')

mapping = {1: 100,
2: 50,
3: 40,
4: 0,
5: 0}
#map out each review score to get the cost of bad review per order
df['cost_of_reviews'] = df['review_score'].map(mapping)
#group by the seller_id and get the sum of their bad reviews
cost = df.groupby('seller_id', as_index=False).agg({'cost_of_reviews':'sum'})
sellers['cost_of_reviews'] = cost['cost_of_reviews']
sellers

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales,share_of_one_stars,share_of_five_stars,review_score,revenues,cost_of_reviews
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000000,218.70,0.333333,0.333333,3.00,341.870,100
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025000,11703.07,0.050000,0.725000,4.55,2290.307,3560
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,1,1,1.000000,158.00,0.000000,1.000000,5.00,15.800,100
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.0,1,1,1.000000,79.99,0.000000,1.000000,5.00,7.999,1110
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,0.0,1,1,1.000000,167.99,1.000000,0.000000,1.00,16.799,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,98dddbc4601dd4443ca174359b237166,sarandi,PR,0.000000,8.230822,2018-07-14 03:10:19,2018-07-23 15:34:03,0.0,2,2,1.000000,158.00,0.000000,1.000000,5.00,15.800,200
2963,f8201cab383e484733266d1906e2fdfa,palhoca,SC,0.000000,13.045797,2017-06-17 15:30:11,2018-08-11 19:55:17,14.0,10,10,1.000000,889.00,0.100000,0.500000,4.10,1208.900,200
2964,74871d19219c7d518d0090283e03c137,sao paulo,SP,0.000000,3.606625,2018-04-05 17:09:46,2018-08-21 19:25:15,5.0,6,7,1.166667,550.04,0.000000,1.000000,5.00,455.004,480
2965,e603cf3fec55f8697c9059638d6c8eb5,pelotas,RS,0.000000,6.657634,2017-02-09 02:43:17,2017-06-16 17:30:16,4.0,8,11,1.375000,297.00,0.000000,0.500000,4.25,349.700,1630


#### Profit

In [6]:
sellers['profits'] = sellers['revenues'] - sellers['cost_of_reviews']
sellers

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales,share_of_one_stars,share_of_five_stars,review_score,revenues,cost_of_reviews,profits
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000000,218.70,0.333333,0.333333,3.00,341.870,100,241.870
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025000,11703.07,0.050000,0.725000,4.55,2290.307,3560,-1269.693
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,1,1,1.000000,158.00,0.000000,1.000000,5.00,15.800,100,-84.200
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.0,1,1,1.000000,79.99,0.000000,1.000000,5.00,7.999,1110,-1102.001
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,0.0,1,1,1.000000,167.99,1.000000,0.000000,1.00,16.799,0,16.799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,98dddbc4601dd4443ca174359b237166,sarandi,PR,0.000000,8.230822,2018-07-14 03:10:19,2018-07-23 15:34:03,0.0,2,2,1.000000,158.00,0.000000,1.000000,5.00,15.800,200,-184.200
2963,f8201cab383e484733266d1906e2fdfa,palhoca,SC,0.000000,13.045797,2017-06-17 15:30:11,2018-08-11 19:55:17,14.0,10,10,1.000000,889.00,0.100000,0.500000,4.10,1208.900,200,1008.900
2964,74871d19219c7d518d0090283e03c137,sao paulo,SP,0.000000,3.606625,2018-04-05 17:09:46,2018-08-21 19:25:15,5.0,6,7,1.166667,550.04,0.000000,1.000000,5.00,455.004,480,-24.996
2965,e603cf3fec55f8697c9059638d6c8eb5,pelotas,RS,0.000000,6.657634,2017-02-09 02:43:17,2017-06-16 17:30:16,4.0,8,11,1.375000,297.00,0.000000,0.500000,4.25,349.700,1630,-1280.300


### 2.2 What-if Analysis

👉 Time to perform the actual analysis, here are our steps:  

1️⃣ Create a method that will help us update the IT Costs after removing sellers along with the products they sold

2️⃣ Sort sellers by increasing profits

3️⃣ Remove sellers one by one, starting from the one with the lowest profit.
- For each number of sellers to remove, compute the financial impact on Olist global profits.

4️⃣ Find an optimal number of sellers to remove that maximizes either Olist's profit margin or profit.

#### Methods

In [8]:
def it_cost(sorted_sellers):
    total_sellers = sellers['seller_id'].nunique()
    total_products =  sellers['quantity'].sum()
    n_sellers = sorted_sellers['seller_id'].nunique()
    n_product = sorted_sellers['quantity'].sum()
    
    accumulate_cost = 500000
    a = 3157.27
    b = 978.23
    updated_cost = accumulate_cost - ((a*((total_sellers - n_sellers)**0.5)) + (b*(total_products - n_product)**0.5))
    return updated_cost

it_cost(sorted_sellers)

500000.0

In [9]:
def olist_financials(sorted_sellers):
    
    updated_revenue = sorted_sellers['revenues'].sum()
    gross_profits = sorted_sellers['profits'].sum()
    system_cost = it_cost(sorted_sellers)
    review_cost = sorted_sellers['cost_of_reviews'].sum()
    
    #computign for gross profit margin
    gpm = (gross_profits/updated_revenue)*100
    
    #computing for the net profit
    np = gross_profits - system_cost
    
    #computing for the net profit margin
    npm = (np/updated_revenue)*100
    
    
    #dictionary to see the financials of the company
    
    financials={'Revenue':updated_revenue, 'IT_Cost': system_cost, 'Gross Profit': gross_profits, 'Gross Profit Margin': gpm,'Net Profit': np, 'Net Profit Margin': npm,}

    return financials

olist_financials(sorted_sellers)

{'Revenue': 2785818.444,
 'IT_Cost': 500000.0,
 'Gross Profit': 1208728.4440000001,
 'Gross Profit Margin': 43.3886295283642,
 'Net Profit': 708728.4440000001,
 'Net Profit Margin': 25.4405826598799}

In [14]:
sorted_sellers = sellers.sort_values(by='profits',ascending=False)

In [20]:
def remove_sellers(sorted_sellers, n):
    updated_sellers = sorted_sellers.drop(sorted_sellers.tail(n).index)
    return olist_financials(updated_sellers)

In [21]:
remove_sellers(sorted_sellers, 10)

{'Revenue': 2777301.404,
 'IT_Cost': 471470.03157611226,
 'Gross Profit': 1568831.404,
 'Gross Profit Margin': 56.48761786317089,
 'Net Profit': 1097361.3724238877,
 'Net Profit Margin': 39.511785463522834}

In [22]:
zero_rem = remove_sellers(sorted_sellers, 0)
ten_rem = remove_sellers(sorted_sellers, 10)
one_hundred_rem = remove_sellers(sorted_sellers, 100)
five_hundred_rem = remove_sellers(sorted_sellers, 500)
eight_hundred_rem = remove_sellers(sorted_sellers, 800)
one_thsnd_rem = remove_sellers(sorted_sellers, 1000)
one_thsnd_two_hun_rem = remove_sellers(sorted_sellers, 1200)

#### Financial Analysis

The more number of unproductive sellers we remove, the company generates higher profit while lowering the IT cost thus making gross profit higher as well as the gross profit margin. 

##### base financials 

    {'Revenue': 2785818.444,
     'IT_Cost': 500000.0,
     'Gross Profit': 1208728.4440000001,
     'Gross Profit Margin': 43.3886295283642,
     'Net Profit': 708728.4440000001,
     'Net Profit Margin': 25.4405826598799}

##### financials when 1 seller was removed:

     'Revenue': 2785358.4820000003,
     'IT_Cost': 493598.30813133065,
     'Gross Profit': 1247668.4820000003,
     'Gross Profit Margin': 44.79382061816774,
     'Net Profit': 754070.1738686697,
     'Net Profit Margin': 27.072643566052463

##### financials when 5 sellers were removed:

    'Revenue': 2783819.889,
    'IT_Cost': 483571.6010949523,
    'Gross Profit': 1381399.8890000002,
    'Gross Profit Margin': 49.622459213631984,
    'Net Profit': 897828.2879050479,
    'Net Profit Margin': 32.25166582984521

##### financials when 30 sellers were removed: 

    'Revenue': 2770178.6739999996,
    'IT_Cost': 457865.9321757384,
    'Gross Profit': 1684648.6740000003,
    'Gross Profit Margin': 60.813718978186046,
    'Net Profit': 1226782.7418242618,
    'Net Profit Margin': 44.28532907781175

##### financials when 100 sellers were removed: 

    'Revenue': 2726075.518,
    'IT_Cost': 421139.08671198494,
    'Gross Profit': 1995435.5180000002,
    'Gross Profit Margin': 73.19810125670921,
    'Net Profit': 1574296.431288015,
    'Net Profit Margin': 57.7495531907717

##### financials when 800 sellers were removed: 


     'Revenue': 2577705.2770000002,
     'IT_Cost': 331601.2953556135,
     'Gross Profit': 2287645.2770000002,
     'Gross Profit Margin': 88.74735592978344,
     'Net Profit': 1956043.9816443867,
     'Net Profit Margin': 75.88315076581917

##### financials when 1000 sellers were removed:

    'Revenue': 2563492.014,
     'IT_Cost': 318021.9803347133,
     'Gross Profit': 2288492.0140000004,
     'Gross Profit Margin': 89.2724456133219,
     'Net Profit': 1970470.033665287,
     'Net Profit Margin': 76.86663437623204

##### financials when 1200 sellers were removed: 

    {'Revenue': 2527184.7060000002,
     'IT_Cost': 302637.1840298213,
     'Gross Profit': 2275634.7060000002,
     'Gross Profit Margin': 90.04623605853683,
     'Net Profit': 1972997.521970179,
     'Net Profit Margin': 78.07096637162772}

#### Optimal number of sellers to remove

- The optimal number of sellers to remove would be around **800-1000 sellers** maintaining a gross profit margin of 88% - 89%. 
- Removing more than 1000 sellers would **decrease the gross profit** despite having a higher gross profit margin due to the lesser IT-cost.

## 3. Investigate other Approaches 🕵️
*(optional)*

- Should Olist remove the worst performing products / categories from its marketplace entirely?
- Should Olist remove only consistently underperforming sellers, after it has a honeymoon period of few months?
- Should Olist restrict seller/customer pairs between certain states to avoid delays?
- Should Olist acquire new sellers, with some cost assumptions to be suggested?
- ...


## Your turn!

🧺 Keep this notebook tidy! 

🗣 📊 You will present your insights to your favorite TA at the end of this `Communicate` unit 💪